In [ ]:
import tensorflow as tf
import copy
from tensorflow.keras.applications.vgg19 import preprocess_input
import scipy.optimize as sopt
vgg19 = tf.keras.applications.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(400,400,3)
)


In [ ]:
import os
import scipy.optimize as sopt
if not os.path.exists('/kaggle/working/nst/'):
    os.mkdir('/kaggle/working/nst/', mode=0o666)

In [ ]:
vgg19.save('/kaggle/working/nst/vgg.keras')

In [ ]:
from tensorflow.keras.layers import AveragePooling2D
custom_objects = {'MaxPooling2D': AveragePooling2D}
mvgg19 = tf.keras.models.load_model('/kaggle/working/nst/vgg.keras', custom_objects = custom_objects)

In [ ]:
style_layers = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]
content_layers = ["block5_conv2"]

In [ ]:
def vgg_layers2model(layer_names):
  outputs = [mvgg19.get_layer(layer_name).output for layer_name in layer_names]
  model = tf.keras.Model(inputs=mvgg19.input,outputs=outputs)
  return model

In [ ]:
nst_model = vgg_layers2model(style_layers+content_layers)

In [ ]:
nst_model.trainable = False

In [ ]:
nst_model.summary()

In [ ]:
nst_model.outputs

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

img_size=400
style_image =  np.array(Image.open("/kaggle/input/imagesfornst/style37.jpg").resize((img_size, img_size)))
#style_image = preprocess_input(style_image)
style_image = tf.constant(np.reshape(style_image, ((1,) + style_image.shape)))

print(style_image.shape)
imshow(style_image[0])
plt.show()

In [ ]:
import copy
content_image_np = np.array(Image.open("/kaggle/input/imagesfornst/content21.jpg").resize((img_size, img_size)))
#content_image = preprocess_input(content_image_np)
content_image = copy.deepcopy(content_image_np)
content_image = tf.constant(np.reshape(content_image, ((1,) + content_image.shape)))
print(content_image.shape)
imshow(content_image[0])
plt.show()

In [ ]:
# def content_cost(orig_img_features, gen_img_features):
#     _, layer_height, layer_width, layer_channels = gen_img_features.shape
#     orig_img_features_unrolled = tf.reshape(orig_img_features, [layer_height * layer_width, layer_channels])
#     gen_img_features_unrolled = tf.reshape(gen_img_features, [layer_height * layer_width, layer_channels])
#     #content_cost = tf.reduce_sum(tf.square(orig_img_features_unrolled - gen_img_features_unrolled))/((4*layer_height*layer_width)**0.5)
#     content_cost = tf.reduce_sum(tf.square(orig_img_features_unrolled - gen_img_features_unrolled))/2   
#     return content_cost

In [ ]:
# def deprocess_img(processes_img):
#   x = processes_img.numpy()
#   if len(x.shape) == 4:
#     x = np.squeeze(x, 0)
#   x[:, :, 0] += 103.939
#   x[:, :, 1] += 116.779
#   x[:, :, 2] += 123.68
#   x = x[:, :, ::-1] # converting BGR to RGB channel
#   x = np.clip(x, 0, 255).astype('uint8')
#   return x

# # imshow(deprocess_img(style_image))
# # plt.show()


In [ ]:
noise_ratio = 0
noise_img = np.random.uniform(0, 255, (1, 400, 400, 3))
white_noise_img = np.random.uniform(0, 255, (1, 400, 400, 3))
content_image_2 = np.reshape(content_image_np, ((1,) + content_image_np.shape))
noise_img = (noise_ratio*noise_img)+((1-noise_ratio)*content_image_2)
#noise_img = preprocess_input(noise_img)
noise_img = tf.Variable(noise_img, dtype='float32')
white_noise_img = tf.Variable(white_noise_img, dtype='float32')

In [ ]:
# noise image should be generated from the content image
# use lbfgs instead of adam

In [ ]:
# def style_cost(style_features, gen_img_features):
#     _, layer_height, layer_width, layer_channels = gen_img_features.shape
#     style_features_unrolled = tf.reshape(style_features, [layer_channels, layer_height * layer_width])
#     gen_img_features_unrolled = tf.reshape(gen_img_features, [layer_channels, layer_height * layer_width])

#     gram_style_matrix = tf.linalg.matmul(style_features_unrolled, style_features_unrolled, transpose_b=True)
#     gram_img_matrix = tf.linalg.matmul(gen_img_features_unrolled, gen_img_features_unrolled, transpose_b=True)

#     return tf.reduce_sum(tf.square(tf.subtract(gram_style_matrix,gram_img_matrix)))/((2*layer_height*layer_width*layer_channels)**2)


In [ ]:
def content_cost(p, x):
  _, h, w, d = p.get_shape()
  M = h*w
  N = d
  K = 0.5
  loss = K * tf.reduce_sum(tf.pow((x - p), 2))
  return loss

def style_cost(a, x):
  _, h, w, d = a.get_shape()
  M = h*w
  N = d
  A = gram_matrix(a, M, N)
  G = gram_matrix(x, M, N)
  loss = (1./(4 * N**2 * M**2)) * tf.reduce_sum(tf.pow((G - A), 2))
  return loss

def gram_matrix(x, area, depth):
  F = tf.reshape(x, (area, depth))
  G = tf.matmul(tf.transpose(F), F)
  return G

In [ ]:
def total_variation_loss(image):
#   x_deltas = image[:, :, 1:, :] - image[:, :, :-1, :]
#   y_deltas = image[:, 1:, :, :] - image[:, :-1, :, :]
#   return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))
  return tf.reduce_sum(tf.image.total_variation(image))

In [ ]:
def clip_0_1(noisy_image):
    """
    Truncate all the pixels in the tensor to be between 0 and 255
    
    Arguments:
    image -- Tensor
    J_style -- style cost coded above

    Returns:
    Tensor
    """
    return tf.clip_by_value(noisy_image, clip_value_min=0, clip_value_max=255)

def tf2img(processes_img):
  x = processes_img.numpy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [ ]:
orig_img_features = nst_model(content_image)[-1]
orig_style_features = nst_model(style_image)[:-1]
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
tf.config.run_functions_eagerly(run_eagerly=True)
alpha = 1
beta = 1
gamma = 1
@tf.function()
def train_step(noise_img):
  with tf.GradientTape() as tape:
    gen_img_features=nst_model(noise_img)
    content_cost_value = content_cost(orig_img_features, gen_img_features[-1])
    style_cost_value = 0
    style_features_list=gen_img_features[:-1]
    style_weights = [0.2,0.2,0.2,0.2,0.2]

    for (i, gen_style_features) in zip(range(5), style_features_list):
      style_cost_value+=(style_weights[i]*style_cost(orig_style_features[i], gen_style_features))
    
    variation_loss = total_variation_loss(noise_img)
        
    total_cost = alpha*content_cost_value + beta*style_cost_value + gamma*variation_loss
    
  grad = tape.gradient(total_cost, noise_img)
  optimizer.apply_gradients([(grad, noise_img)]) 
  noise_img.assign(clip_0_1(noise_img))
  return (content_cost_value), (style_cost_value), (variation_loss)

In [ ]:
content_cost_value, style_cost_value, variation_loss = train_step(white_noise_img)
total_cost = alpha*content_cost_value + beta*style_cost_value + gamma*variation_loss
average_cost = total_cost/3
alpha = average_cost/content_cost_value
beta = 100*average_cost/style_cost_value
gamma = 0.1*average_cost/variation_loss
print(f'alpha is {alpha}\nbeta is {beta}\ngamma is {gamma}')

In [ ]:
img = tf2img(noise_img)
imshow(img)
plt.show()

In [ ]:
orig_img_features = nst_model(content_image)[-1]
orig_style_features = nst_model(style_image)[:-1]
tf.config.run_functions_eagerly(run_eagerly=True)

@tf.function
def compute_loss_and_grads(noise_img):
    with tf.GradientTape() as tape:
        tape.watch(noise_img)
        
        gen_img_features = nst_model(noise_img)
        content_cost_value = content_cost(orig_img_features, gen_img_features[-1])
        
        style_cost_value = 0
        style_features_list = gen_img_features[:-1]
        style_weights = [0.2, 0.2, 0.2, 0.2, 0.2]
        
        for i, gen_style_features in zip(range(5), style_features_list):
            style_cost_value += (style_weights[i] * style_cost(orig_style_features[i], gen_style_features))
        
        variation_loss = total_variation_loss(noise_img)
        
        total_cost = alpha * content_cost_value + beta * style_cost_value + gamma * variation_loss
        print(f"content_cost: {alpha*content_cost_value}, style_cost is {beta*style_cost_value}, variation_loss is {gamma*variation_loss}")
    grads = tape.gradient(total_cost, noise_img)
    return total_cost, grads

# Wrapper function for SciPy optimizer
def func(x):
    noise_img = tf.Variable(x.reshape(content_image.shape), dtype=tf.float32)
    #noise_img = clip_0_1(noise_img)
    loss, grads = compute_loss_and_grads(noise_img)
    return loss.numpy().astype(np.float64), grads.numpy().flatten().astype(np.float64)


# Run the L-BFGS-B optimization
result = sopt.minimize(fun=func, x0=noise_img.numpy().flatten(), jac=True, method='L-BFGS-B', options={'maxiter': 7000})

noise_img =  tf.Variable(result.x.reshape(content_image.shape), dtype=tf.float32)
noise_img = clip_0_1(noise_img)
img = tf2img(noise_img)
imshow(img)
plt.show()

In [ ]:
# content_cost_value, style_cost_value, variation_loss = train_step(noise_img)
# print(f"content cost: {content_cost_value}, style cost: {style_cost_value}, variation cost: {variation_loss}")
# img = tf2img(noise_img)
# imshow(img)
# plt.show()

In [ ]:
# noise_img.numpy().min()

In [ ]:
# gamma = 16000

In [ ]:
# for i in range(1001):
#     try:
#       content_cost_value, style_cost_value, variation_loss = train_step(noise_img)
#     except Exception as ex:
#       print(ex)
#     if(i==20000):
#          optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
#     if(i==40000):
#          optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# #     if i%100 == 0:
# #           print(f"content cost: {alpha*content_cost_value}, style cost: {beta*style_cost_value}")
#     if i%1000 == 0:
#         img=tf2img(noise_img)
#         imshow(img)
#         plt.show()

In [ ]:
# image_to_save = Image.fromarray(img)
# image_to_save.save('/kaggle/working/deprocessed_style_image.png')

In [ ]:
#         img=tf2img(noise_img)
#         imshow(img)
#         plt.show()

In [ ]:
noise_img =  tf.Variable(result.x.reshape(content_image.shape), dtype=tf.float32)
noise_img = clip_0_1(noise_img)
arr = tf2img(noise_img)
from PIL import Image
im = Image.fromarray(arr)
im.save("/kaggle/working/result3.png")


In [ ]:
noise_img =  tf.Variable(result.x.reshape(content_image.shape), dtype=tf.float32)
noise_img = clip_0_1(noise_img)
img = tf2img(noise_img)
imshow(img)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # figsize controls the overall size of the figure

# Display each image in a subplot
content_image = tf2img(content_image)
style_image = tf2img(style_image)

axes[0].imshow(content_image)
axes[0].axis('off')  # Hide the axis
axes[0].set_title('Content') 

axes[1].imshow(style_image)
axes[1].axis('off')  # Hide the axis
axes[1].set_title('Style')

axes[2].imshow(img)
axes[2].axis('off')  # Hide the axis
axes[2].set_title('Result Image')

plt.savefig('/kaggle/working/result2.png', bbox_inches='tight', pad_inches=0.1)
# Display the figure
plt.show()

In [ ]:
#plt.savefig('/kaggle/working/result2.png', bbox_inches='tight', pad_inches=0.1)